# Setup phone calibration plane

In [ ]:
from time import sleep
import sys

from ipywidgets import Box
from IPython.display import display

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.devices.robot.robot_on_plane import RobotOnPlane
from skrgbd.devices.rig import Rig

In [ ]:
robot = RobotOnPlane()

rig = Rig(stl_right=False, tis_left=False, tis_right=False, realsense=False, kinect=False,
          phone_left=True, phone_right=True)
rig.init_cameras().join()
rig.start_cameras().join()

for camera in rig.phones:
    camera.setup('room_lights', 'calibration')

In [ ]:
widget = Box([phone.start_streaming('ir', ticks=False) for phone in rig.phones])

1. Position the robot at middle right of the calibration plane.

In [ ]:
right = 1
robot.move_to((right, .5), .1);

2. Position the calibration board
    * at the working distance of the cameras,
    * so that the tag is slightly beyond the left boundary of both IR images,
    * so that the board is parallel to the image planes.

Make sure that the white area around the tag is fully visible on the images.

In [ ]:
widget

3. Find the vertical position of the robot so that the tag is at the top boundary of all the images.

In [ ]:
bottom = .21
robot.move_to((right, bottom), .1);
widget

4. Find the vertical position of the robot so that the tag is at the bottom boundary of all the images.

In [ ]:
top = .75
robot.move_to((right, top), .1);
widget

5. Find the horizontal position of the robot so that the tag is at the right boundary of all the images.

In [ ]:
left = .36
robot.move_to((left, top), .1);
widget

6. Fine tune the boundaries so that the tag is fully visible at the corner positions.

In [ ]:
widget

In [ ]:
left = .33
right = .98
bottom = .21
top = .72

robot.move_to((left, top), .1);
robot.move_to((right, top), .1);
robot.move_to((right, bottom), .1);
robot.move_to((left, bottom), .1);
robot.move_to((left, top), .1);

In [ ]:
rig.phone_left.stop_streaming('ir')
rig.phone_right.stop_streaming('ir')

## Find tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

from pupil_apriltags import Detector
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration_helper.trajectories.phone_plane import PhoneCalibrationPlane, make_im_slice
from skrgbd.devices.rig import Rig
from skrgbd.devices.robot.robot_on_plane import RobotOnPlane

In [ ]:
robot = RobotOnPlane()
trajectory = PhoneCalibrationPlane(robot)
trajectory.move_zero(.1)

In [ ]:
rig = Rig(stl_right=False, tis_left=False, tis_right=False, realsense=False, kinect=False,
          phone_left=True, phone_right=True)
rig.init_cameras().join()
rig.start_cameras().join()

for camera in rig.phones:
    camera.setup('room_lights', 'calibration')

In [ ]:
detector = Detector(
    families='tag36h11',
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25,
    debug=0
)

In [ ]:
image = rig.phone_right.get_ir()

image = np.clip((image.astype(np.float32) / image.max() * 255).round(), 0, 255).astype(np.uint8)
plt.imshow(image, cmap='gray')

In [ ]:
detection = detector.detect(image)[0]

x, y = detection.center
size = (detection.corners[0, 0] - detection.center[0]) * 2

size = size * 1.3
size = round(size)
x = round(x)
y = round(y)

print(f'({y}, {x}), {size}')
plt.imshow(image[make_im_slice([y, x], size)], cmap='gray')

## Check tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration_helper.trajectories.phone_plane import PhoneCalibrationPlane
from skrgbd.devices.rig import Rig

In [ ]:
rig = Rig(stl_right=False, tis_left=False, tis_right=False, realsense=False, kinect=False,
          phone_left=True, phone_right=True)
rig.init_cameras().join()
rig.start_cameras().join()

for camera in rig.phones:
    camera.setup('room_lights', 'calibration')

In [ ]:
trajectory = PhoneCalibrationPlane(None)
trajectory.stream_tag(rig.phone_left, rig.phone_right)

In [ ]:
trajectory.stop_tag_streaming()